In [ ]:
import torch
from torch2trt import torch2trt
from torchvision.models.alexnet import alexnet

# create some regular pytorch model...
model = alexnet(pretrained=True).eval().cuda()

# create example data
x = torch.ones((1, 3, 224, 224)).cuda()

# convert to TensorRT feeding sample data as input
model_trt = torch2trt(model, [x])

torch.save(model_trt.state_dict(), 'alexnet_trt.pth')


In [ ]:
import torch
from torch2trt import TRTModule

model_trt = TRTModule()
model_trt.load_state_dict(torch.load('alexnet_trt.pth'))
device = torch.device('cuda')
model = model_trt.to(device)

In [ ]:
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np

mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

normalize = torchvision.transforms.Normalize(mean, std)

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

In [ ]:
# Import Pillow
import torchvision.transforms as transforms

from torchvision import transforms
transform = transforms.Compose([            #[1]
 transforms.Resize(256),                    #[2]
 transforms.CenterCrop(224),                #[3]
 transforms.ToTensor(),                     #[4]
 transforms.Normalize(                      #[5]
 mean=[0.485, 0.456, 0.406],                #[6]
 std=[0.229, 0.224, 0.225]                  #[7]
 )])

from PIL import Image
img = Image.open("dog.jpg")
img_t = transform(img)
batch_t = torch.unsqueeze(img_t, 0)
model_trt.eval()
out = model_trt(batch_t)
print(out.shape)

with open('imagenet_classes.txt') as f:
  classes = [line.strip() for line in f.readlines()]

_, index = torch.max(out, 1)

percentage = torch.nn.functional.softmax(out, dim=1)[0] * 100

print(labels[index[0]], percentage[index[0]].item())

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets

from jetbot import Camera, bgr8_to_jpeg

image = widgets.Image(format='jpeg', width=224, height=224)
from PIL import Image
img = Image.open("dog.jpg")
print(img)

camera = Camera.instance()
type(camera)
#camera_link = traitlets.dlink((img, 'value'), (image, 'value'), transform=bgr8_to_jpeg)



In [ ]:
import ipywidgets.widgets as widgets
image = widgets.Image(format='jpeg', width=300, height=300)

display(image)

In [18]:
import ipywidgets.widgets as widgets

file = open("dog.jpg", "rb")
image = file.read()
widgets.Image(
    value=image,
    format='jpg',
    width=300,
    height=400,
)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00`\x00`\x00\x00\xff\xdb\x00\x84\x00\x08\x06\x06\x…